## **Define una estructura de datos**

In [1]:
import os
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


In [2]:
# Librerías relevantes
import pandas as pd
import numpy as np
import json
import re

In [3]:
# Asigna un tipo de datos adecuado a cada columna en el orden en que aparecen en la tabla original,
# Los nombres de algunas columnas se cambiaron respecto al original, según se detalla en 
# ENLACE A ARCHIVO
# para asegurar una mayor legibilidad y para posibilitar el cruce con la tabla de seguimiento nutricional
dtypes_sociodemo = {
    'Cod_clase': 'category',
    'Ind_grupo_sisben_4': 'category',
    'Ind_nivel_sisben_4': 'category',
    'Tip_vivienda': 'category',
    'Ind_tiene_energia': 'category',
    'Ind_tiene_alcantarillado': 'category',
    'Ind_tiene_gas': 'category',
    'Ind_tiene_recoleccion': 'category',
    'Ind_tiene_acueducto': 'category',
    'n_hogares_vivienda': 'UInt8',
    'Tip_ocupa_vivienda': 'category',
    'Ind_tiene_cocina': 'category',
    'Ind_tiene_nevera': 'category',
    'Ind_tiene_lavadora': 'category',
    'Ind_tiene_pc': 'category',
    'Ind_tiene_internet': 'category',
    'n_personas_hogar': 'UInt8',
    'Sexo': 'category',
    'Ind_ninguna_discapac': 'category',
    'Tip_seg_social': 'category',
    'Ind_acudio_salud': 'category',
    'Ind_fue_atendido_salud': 'category',
    'Tip_cuidado_ninos': 'category',
    'Ind_recibe_comida': 'category',
    'Ind_leer_escribir': 'category',
    'Ind_estudia': 'category',
    'privado_logro_educ': 'category',
    'privado_analfabe': 'category',
    'privado_inasistencia': 'category',
    'privado_rezago': 'category',
    'privado_primera_inf': 'category',
    'privado_trabajo_inf': 'category',
    'privado_desemp_largo': 'category',
    'privado_trabajo_info': 'category',
    'privado_aseguram': 'category',
    'privado_acceso_salud': 'category',
    'privado_agua': 'category',
    'privado_excreta': 'category',
    'privado_suelo': 'category',
    'privado_pared': 'category',
    'privado_hacina': 'category',
    'n_privaciones': 'float32',
    'IndicadorPobrezaMulti': 'category',
    'ingresos_promP_imp': 'float32',
    'gasto_ppers_imp': 'float32',
    'gasto_alim_ppers_imp': 'float32',
    'porc_gasto_alim': 'float32',
    'union_temprana': 'category',
    'n_ninos': 'UInt8',
    'Tip_hijo': 'category',
    'Jefat_fem': 'category',
    'Tip_familia': 'category',
    'Genera_ingresos': 'category',
    'Nivel_educ_padre': 'category',
    'Nivel_educ_madre': 'category',
    'anno_encuesta': 'category',
    'Uni_dias_agua': 'float32',
    'Ind_afec_evento_natural': 'category',
    'n_afec_evento_natural': 'UInt8',
    'Edad_padres_mayor': 'UInt8',
    'Edad_padres_menor': 'UInt8',
    'Comparte_cocina_sanitario': 'category',
    'Estrato': 'category',
    'cod_mpio': 'category',
    'cod_dpto': 'category',
    'FechaNacimiento': 'datetime64[ns]',
    'IdBeneficiario': 'UInt32',
    'Id': 'UInt32'
    }
# Guarda el diccionario de tipos de datos
with open('datatypes_dictionaries/dtypes_sociodemo.json', 'w') as file:
    json.dump(dtypes_sociodemo, file)

In [4]:
# Especifica las categorías válidas (ordenadas) de las variables categóricas (ordinales)
soc_cat = {
  'Ind_grupo_sisben_4': list('ABCD'),
  'Ind_nivel_sisben_4': list(range(1, 52)),
  'Nivel_educ_padre': list(range(10)),
  'Nivel_educ_madre': list(range(10)),
  'anno_encuesta': list(range(2017, 2030)),
  'Estrato': list(range(7))
  }

# Lista de variables categóricas ordinales
soc_ord = list(soc_cat.keys())

# Asigna a cada código numérico un diccionario con el significado de cada categoría
soc_maps = {
  'Cod_clase': {
      1: 'Cabecera',
      2: 'Centro poblado',
      3: 'Rural disperso'
      },
  'Tip_vivienda': {
      1: 'Casa',
      2: 'Apartamento',
      3: 'Cuarto',
      4: 'Otro tipo de vivienda',
      5: 'Vivienda indígena'
      },
  'Tip_ocupa_vivienda': {
      1: 'En arriendo o subarriendo',
      2: 'Propia, la están pagando',
      3: 'Propia, totalmente pagada',
      4: 'Con permiso del propietario',
      5: 'Posesión sin título, ocupante de hecho'
      },
  'Sexo': {
      0: 'F',
      1: 'M'
      },
  'Tip_seg_social': {
      0: 'No sabe',
      1: 'Contributivo',
      2: 'Especial (Fuerzas Armadas, Ecopetrol, universidades públicas, magisterio)',
      3: 'Subsidiado (EPS-S)',
      9: 'Ninguna'
      },
  'Tip_cuidado_ninos': {
      1: 'Asiste a un lugar comunitario, jardín o centro de desarrollo infantil o colegio',
      2: 'Con su padre o madre en la casa',
      3: 'Con su padre o madre en el trabajo',
      4: 'Con empleada o niñera en la casa',
      5: 'Al cuidado de un pariente de 18 años o más',
      6: 'Al cuidado de un pariente menor de 18 años',
      7: 'En casa solo',
      8: 'No responde',
      9: 'No aplica por flujo'
      },
  'Tip_hijo': {
      1: 'Mayor',
      2: 'Único',
      3: 'Otro'
      },
  'Tip_familia': {
      1: 'Sin madre y sin padre',
      2: 'Solo madre y cónyuge',
      3: 'Solo padre y cónyuge',
      4: 'Solo madre',
      5: 'Solo padre',
      6: 'Con madre y con padre'
      },
  'Genera_ingresos': {
      1: 'Solo figura paterna',
      2: 'Solo figura materna',
      3: 'Ninguno',
      4: 'Ambos'
      },
  'Nivel_educ_padre': {
      0: 'Ninguno',
      1: 'Preescolar',
      2: 'Básica primaria',
      3: 'Básica secundaria (6 a 9)',
      4: 'Media (10 a 13)',
      5: 'Tecnológica o técnico',
      6: 'Universitario',
      7: 'Postgrado',
      9: 'No aplica por flujo'
      },
  'Nivel_educ_madre': {
      0: 'Ninguno',
      1: 'Preescolar',
      2: 'Básica primaria',
      3: 'Básica secundaria (6 a 9)',
      4: 'Media (10 a 13)',
      5: 'Tecnológica o técnico',
      6: 'Universitario',
      7: 'Postgrado',
      9: 'No aplica por flujo'
      }
  }

# Lista de variables categóricas binarias (categorías válidas: [0, 1])
soc_bin = [
  'Ind_tiene_energia',
  'Ind_tiene_alcantarillado',
  'Ind_tiene_gas',
  'Ind_tiene_recoleccion',
  'Ind_tiene_acueducto',
  'Ind_tiene_cocina',
  'Ind_tiene_nevera',
  'Ind_tiene_lavadora',
  'Ind_tiene_pc',
  'Ind_tiene_internet',
  'Ind_ninguna_discapac',
  'Ind_acudio_salud',
  'Ind_fue_atendido_salud',
  'Ind_recibe_comida',
  'Ind_leer_escribir',
  'Ind_estudia',
  'privado_logro_educ',
  'privado_analfabe',
  'privado_inasistencia',
  'privado_rezago',
  'privado_primera_inf',
  'privado_trabajo_inf',
  'privado_desemp_largo',
  'privado_trabajo_info',
  'privado_aseguram',
  'privado_acceso_salud',
  'privado_agua',
  'privado_excreta',
  'privado_suelo',
  'privado_pared',
  'privado_hacina',
  'IndicadorPobrezaMulti',
  'union_temprana',
  'Jefat_fem',
  'Ind_afec_evento_natural',
  'Comparte_cocina_sanitario'
  ]

# Define el conjunto de categorías válidas para variables categóricas binarias y no ordinales
for col in soc_bin:
  soc_cat[col] = [0, 1]
for col in soc_maps:
  soc_cat[col] = list(soc_maps[col].keys())

# Guarda el diccionario de categorías válidas
with open('datatypes_dictionaries/soc_cat.json', 'w') as file:
    json.dump(soc_cat, file)

# Guarda el diccionario de significados de cada categoría
with open('datatypes_dictionaries/soc_maps.json', 'w') as file:
    json.dump(soc_maps, file)

In [5]:
# Abre y lee el archivo con los nombres de cada columna en el orden en que aparecen en la tabla original
# Los nombres de algunas columnas se cambiaron respecto al original, según se detalla en 
# ENLACE A ARCHIVO
# para asegurar una mayor legibilidad y para posibilitar el cruce con la tabla de seguimiento nutricional
with open('datatypes_dictionaries/col_names_tomas.txt', 'r') as file:
    col_names = [line.strip() for line in file.readlines()]

# Columnas que identifican al beneficiario
# Se encuentran una única vez en la tabla original
dtypes_tomas = {
    'IdToma': 'UInt32', # Variable agregada no presente en la tabla original No deberá estar aquí :(
    'Registro': 'UInt32', # Variable agregada no presente en la tabla original 
    'Vigencia': 'category',
    'IdBeneficiario': 'UInt32',
    'Id': 'UInt32',
    'Tipo': 'category',
    'FechaNacimiento': 'datetime64[ns]',
    'Sexo': 'category',
    'CodigoPaisNacimiento': 'category',
    'PaisNacimiento': 'category',
    'PresentaDiscapacidad': 'category',
    'ZonaUbicacion': 'category',
    'GrupoEtnico': 'category',
    'MesesLactanciaMaternaExclusiva': 'UInt8',
    'MesesLactanciaMaternaTotal': 'UInt8',
    'PesoAlNacer': 'float32',
    'TallaAlNacer': 'float32',
    'AntecedentePremadurez': 'category',
    'EdadGestacionalAlNacer': 'UInt8',
    'RegimenSeguridadSocial': 'category'
    }

# Columnas que identifican a cada toma nutricional 
# Algunas se repiten con sufijos T1, ..., T4 y T1N, ..., T12N, para cada toma nutricional
dtypes_vars_toma = {
  'IdToma': 'UInt32',
  'Registro': 'UInt32',
  'Vigencia': 'category',
  'IdBeneficiario': 'UInt32',
  'Id': 'UInt32',
  'Toma': 'category',
  'Servicio': 'category',
  'FechaValoracionNutricional': 'datetime64[ns]',
  'EdadMeses': 'float32',
  'FechaMedicionPerimetroBraquial': 'datetime64[ns]',
  'MedicionPerimetroBraquial': 'float32',
  'Peso': 'float32',
  'Talla': 'float32',
  'ZScoreTallaEdad': 'float32',
  'ZScorePesoEdad': 'float32',
  'ZScorePesoTalla': 'float32',
  'ZScoreIMC': 'float32',
  'EstadoTallaEdad': 'category',
  'EstadoPesoEdad': 'category',
  'EstadoPesoTalla': 'category',
  'EstadoIMC': 'category',
  'Flag': 'UInt8',
  'FechaRegistroSaludNutricion': 'datetime64[ns]',
  'PresentaCarneVacunacion': 'category',
  'ControlesCrecimDesarrollo': 'UInt8',
  'AntecedentePremadurez': 'category',
  'Direccion': 'category' # Variable nueva
  }

# Asigna a cada columna el tipo de datos adecuado de acuerdo a su nombre sin sufijo
for col in col_names:
  # Nombre de la variable sin sufijo, si lo tiene
  variable = re.sub('T*\d+N*', '', col)
  if variable in dtypes_vars_toma.keys():
    dtypes_tomas[col] = dtypes_vars_toma[variable]

# Guarda el diccionario de tipos de datos para todas las columnas        
with open('datatypes_dictionaries/dtypes_tomas.json', 'w') as file:
    json.dump(dtypes_tomas, file)

# Guarda el diccionario de tipos de datos para las variables por toma nutricional
with open('datatypes_dictionaries/dtypes_vars_toma.json', 'w') as file:
    json.dump(dtypes_vars_toma, file)

In [6]:
# Especifica las categorías válidas (ordenadas) de las variables categóricas (ordinales)
tom_cat = {
  'Vigencia': list(range(2017, 2020)),
  'Toma': [f'{n + 1}' for n in range(4)] + [f'{n + 1}N' for n in range(12)],
  'EstadoTallaEdad': [
    'Retraso en talla',
    'Riesgo de baja talla',
    'Talla adecuada para la edad',
    'Talla por encima de lo esperado'
    ],
  'EstadoPesoEdad': [
    'Desnutrición global severa',
    'Desnutrición global',
    'Riesgo de peso bajo para la edad',
    'Peso adecuado para la edad',
    'Sobrepeso',
    'Obesidad'
    ],  
  'EstadoPesoTalla': [
    'Desnutrición aguda severa',
    'Desnutrición aguda moderada',
    'Riesgo de desnutrición aguda',
    'Peso adecuado para la talla',
    'Riesgo de sobrepeso',
    'Sobrepeso',
    'Obesidad'
    ],
  'EstadoIMC': [
    'Delgadez',
    'Riesgo para la delgadez',
    'Adecuado para la edad',
    'Riesgo de sobrepeso',
    'Sobrepeso',
    'Obesidad'
    ],
  'Tipo': [
    'MENOR DE SEIS MESES',
    'NIÑO O NIÑA ENTRE 6 MESES Y 5 AÑOS Y 11 MESES',
    'ESCOLAR O ADOLESCENTE', 
    'MUJER GESTANTE',
    'NIÑOS, NIÑAS, ADOLESCENTES O ADULTOS'
    ],
  'Sexo': ['F', 'M'],
  'PresentaDiscapacidad': ['NO', 'SI'],
  'ZonaUbicacion': ['CABECERA', 'RESTO'],
  'GrupoEtnico': [
    'AFROCOLOMBIANO',
    'COMUNIDAD NEGRA',
    'PALENQUERO',
    'INDIGENA',
    'RROM/GITANO',
    'RAIZAL DEL ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA',
    'NO SE AUTORECONOCE EN NINGUNO DE LOS ANTERIORES'
    ],
  'AntecedentePremadurez': ['N', 'S'],
  'RegimenSeguridadSocial': [
    'BENEFICIARIO DEL REGIMEN SUBSIDIADO ',
    'BENEFICIARIO O COTIZANTE DEL  REGIMEN CONTRIBUTIVO',
    'BENEFICIARIO DEL RÉGIMEN ESPECIAL',
    'NO AFILIADO(A)'
    ],
  'Servicio': [
    'ATENCIÓN A NIÑOS HASTA LOS 3 AÑOS EN ESTABLECIMIENTOS DE RECLUSIÓN A MUJERES INTEGRAL',
    'ATENCIÓN PROPIA E INTERCULTURAL',
    'CDI CON ARRIENDO - INSTITUCIONAL INTEGRAL',
    'CDI SIN ARRIENDO -  INSTITUCIONAL INTEGRAL',
    'CENTROS DE RECUPERACIÓN NUTRICIONAL PARA LA PRIMERA INFANCIA ',
    'DESARROLLO INFANTIL EN MEDIO FAMILIAR - FAMILIAR INTEGRAL',
    'DESARROLLO INFANTIL EN MEDIO FAMILIAR CON ARRIENDO - FAMILIAR INTEGRAL',
    'DESARROLLO INFANTIL EN MEDIO FAMILIAR SIN ARRIENDO - FAMILIAR INTEGRAL',
    'ESTRATEGIAS DE DESARROLLO ALIMENTARIO O NUTRICIONAL (MODALIDAD MIL DÍAS PARA CAMBIAR EL MUNDO)',
    'HCB  AGRUPADOS -INSTITUCIONAL TRADICIONAL',
    'HCB FAMI-FAMILIAR TRADICIONAL',
    'HCB INTEGRAL -COMUNITARIO  INTEGRAL',
    'HCB TRADICIONAL- COMUNITARIO (T)',
    'HOGARES EMPRESARIALES - INSTITUCIONAL INTEGRAL',
    'HOGARES INFANTILES - INSTITUCIONAL INTEGRAL',
    'HOGARES MULTIPLES - INSTITUCIONAL INTEGRAL',
    'JARDINES SOCIALES - INSTITUCIONAL INTEGRAL',
    'SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA  - COMUNITARIO INTEGRAL',
    'SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - ATENCIÓN PROPIA E INTERCULTURAL',
    'SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - FAMILIAR INTEGRAL',
    'SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - GRADO TRANSICIÓN CON ATENCIÓN INTEGRAL',
    'SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - INSTITUCIONAL INTEGRAL'
    ],
  'PresentaCarneVacunacion': ['N', 'S'],
  'Direccion': ['Primera Infancia', 'Nutrición'],
  }

# Lista de variables categóricas ordinales
tom_ord = list(tom_cat.keys())[:5]

# Guarda el diccionario de categorías válidas
with open('datatypes_dictionaries/tom_cat.json', 'w') as file:
    json.dump(tom_cat, file)

In [7]:
# Guarda las listas de variables categóricas ordinales en ambas tablas
cols_ord = {
    'soc': soc_ord,
    'tom': tom_ord
    }
with open('datatypes_dictionaries/cols_ord.json', 'w') as file:
    json.dump(cols_ord, file)